<a href="https://colab.research.google.com/github/fyuuki0jp/colab_llm_training/blob/main/sft_plamo2_1b_instruct_ver0_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# dependence install

In [ ]:
%%writefile requirements.txt
causal-conv1d==1.5.0.post8
fastrlock==0.8.3
mamba-ssm==2.2.4
numba==0.61.0
numba-cuda==0.0.17.1
sentence-transformers==3.4.1
torch==2.4.1+cu124
torchinfo
torchaudio==2.4.1+cu124
torchsummary==1.5.1
torchvision==0.19.1+cu124
transformers==4.48.2
trl==0.14.0
datasets
deepspeed>=0.9.3
mpi4py

Writing requirements.txt


In [ ]:
!pip install -Uq torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --index-url https://download.pytorch.org/whl/cu124
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.4/883.4 kB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
import datasets
import string
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

# モデルのセットアップ

In [ ]:
base_model = "pfnet/plamo-2-1b"
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(base_model, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenization_plamo.py:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/pfnet/plamo-2-1b:
- tokenization_plamo.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer.jsonl:   0%|          | 0.00/3.32M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

...

model-00004-of-00007.safetensors:   0%|          | 0.00/621k [00:00<?, ?B/s]

model-00005-of-00007.safetensors:   0%|          | 0.00/132k [00:00<?, ?B/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
# @title 特殊トークンでtokenizerを拡張する
# 拡張するカスタムトークン
custom_tokens = [
    "<start_of_turn>", # gemmaのpromptフォーマットを参考にするので
    "<end_of_turn>", # gemmaのpromptフォーマットを参考にするので
    "<think>", # モデルが利用するCoTの開始トークン
    "</think>", # モデルが利用するCoTの終了トークン
    "<documents>", #モデルへ参考文献の開始を示す特殊トークン
    "</documents>", #モデルへ参考文献の終わりを示す特殊トークン
    "<content>", # 参考文献の1件の開始を示す特殊トークン
    "</content>", # 参考文献の1件の終了を示す特殊トークン
    "<tool_def>", # モデルが回答に利用できるtoolの定義開始トークン
    "</tool_def>", # モデルが回答に利用できるtoolの定義開始トークン
    "<tool_call>", # モデルが回答時にtoolの呼び出し開始であることを明示するトークン
    "</tool_call>", # モデルが回答時にtoolの呼び出しが完了したことを明示するトークン
]

print(f"拡張後の語彙サイズ：{len(tokenizer)}")
tokenizer.add_special_tokens({"additional_special_tokens": custom_tokens})
print(f"拡張後の語彙サイズ：{len(tokenizer)}")

拡張後の語彙サイズ：100000
拡張後の語彙サイズ：100012


In [ ]:
# @title チャットテンプレートを定義する (過去のやり取りを取り扱う際は\<think>\</think>の間は渡さないようにする)
tokenizer.chat_template = """{{bos_token}}{%- for message in messages %}
    {%- if message['role'] == 'system' %}
        {{- '<start_of_turn>system\n' + message['content'].strip() + '\n<end_of_turn>\n' }}
    {%- elif message['role'] == 'user' %}
        {{- '<start_of_turn>user\n' + message['content'].strip() +'\n'}}
        {%- if tools %}
            {%- for tool in tools %}
                {{- '<tool_def>' + tool['function']['name'] }}
                {%- for argument in tool['function']['parameters']['properties'] %}
                    {{- '\n'+argument + ': ' + tool['function']['parameters']['properties'][argument]['description'] }}
                {%- endfor %}
                {{'\n</tool_def>\n'}}
            {%- endfor %}
        {%- endif %}
        {%- if documents %}
            {%- for document in documents %}
                {{- '<context>' + document['no']+'\n' }}
                {{- document['text'].strip() }}
                {{- '\n</context>\n'}}
            {%- endfor %}
        {%- endif %}
        {{- '<end_of_turn>\n'}}
    {%- elif message['role'] == 'assistant' %}
        {{- '<start_of_turn>model\n'  + message['content'].split('</think>')[-1] + '\n<end_of_turn>\n' }}
    {%- endif %}
{%- endfor %}
{%- if messages[-1]['role']=='user' %}
  {{- '<start_of_turn>model\n' }}
{%- endif %}
"""

In [ ]:
# @title チャットテンプレートのテスト(通常の会話、モデルが必要に応じてCoTする)
messages = [
    {"role": "system", "content": "あなたは優秀なアシスタントです。"},
    {"role": "user", "content": "A～Dの中で3人は正直者で、1人が嘘つきです。嘘つきはだれでしょう？"},
]
print(tokenizer.apply_chat_template(messages, tokenize=False))

<|plamo:bos|><start_of_turn>system
あなたは優秀なアシスタントです。
<end_of_turn>
<start_of_turn>user
A～Dの中で3人は正直者で、1人が嘘つきです。嘘つきはだれでしょう？
<end_of_turn>
<start_of_turn>model



In [ ]:
# @title チャットテンプレートのテスト(RAG)
messages = [
    {"role": "system", "content": "あなたは優秀なアシスタントです。"},
    {"role": "user", "content": "富士山について教えて"},
]

documents = [
    {"no":"1","text":"富士山は日本の最高峰で、美しい円錐形の火山として世界的に有名です。山梨県と静岡県に跨り、日本の象徴とされています。登山シーズンは7月～9月で、多くの登山客で賑わいます。"},
    {"no":"2","text":"富士山は玄武岩質の成層火山で、標高3,776m。山体は直径約50km、体積約1,400km³。約10万年前から活動を開始し、複数の火山体から構成されています。特別天然記念物及び日本の名水百選に選定。"}
]

print(tokenizer.apply_chat_template(messages,documents=documents, tokenize=False))

<|plamo:bos|><start_of_turn>system
あなたは優秀なアシスタントです。
<end_of_turn>
<start_of_turn>user
富士山について教えて
<context>1
富士山は日本の最高峰で、美しい円錐形の火山として世界的に有名です。山梨県と静岡県に跨り、日本の象徴とされています。登山シーズンは7月～9月で、多くの登山客で賑わいます。
</context>
<context>2
富士山は玄武岩質の成層火山で、標高3,776m。山体は直径約50km、体積約1,400km³。約10万年前から活動を開始し、複数の火山体から構成されています。特別天然記念物及び日本の名水百選に選定。
</context>
<end_of_turn>
<start_of_turn>model



In [ ]:
# @title チャットテンプレートのテスト(ツール呼び出し)
messages = [
    {"role": "system", "content": "あなたは優秀なアシスタントです。"},
    {"role": "user", "content": "明日の東京の天気を教えて"},
]

tool = [
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    },
                },
                "required": ["location", "format", "num_days"],
            },
        },
    }
]

print(tokenizer.apply_chat_template(messages,tools=tool, tokenize=False))

<|plamo:bos|><start_of_turn>system
あなたは優秀なアシスタントです。
<end_of_turn>
<start_of_turn>user
明日の東京の天気を教えて
<tool_def>get_n_day_weather_forecast
location: The city and state, e.g. San Francisco, CA
format: The temperature unit to use
num_days: The number of days to forecast                
</tool_def>
<end_of_turn>
<start_of_turn>model



In [ ]:
# @title tokenizerの拡張に合わせてモデルも修正する
model.resize_token_embeddings(len(tokenizer))
# モデル構造の可視化
from torchinfo import summary
summary(model)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Layer (type:depth-idx)                                  Param #
PlamoForCausalLM                                        --
├─PlamoModel: 1-1                                       --
│    └─Embedding: 2-1                                   204,824,576
│    └─PlamoDecoder: 2-2                                --
│    │    └─ModuleList: 3-1                             1,086,639,872
│    └─RMSNorm: 2-3                                     2,048
├─Linear: 1-2                                           204,824,576
Total params: 1,496,291,072
Trainable params: 1,496,291,072
Non-trainable params: 0

# データセットの準備

In [ ]:
multitask_reason_ja = datasets.load_dataset('fyuuki0jp/plamo2-1b-instruct-dataset-6k',split='train')
reason_math_ja = datasets.load_dataset('fyuuki0jp/openr1-math-instruct-4k',split='train')
WikiHowNFQA_ja = datasets.load_dataset('GENIAC-Team-Ozaki/WikiHowNFQA-ja_cleaned',split='train')
answerCarefully = datasets.load_dataset('llm-jp/AnswerCarefully','v2.0',split='dev')

README.md:   0%|          | 0.00/456 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6327 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/886 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/67.5M [00:00<?, ?B/s]

...

train-00000-of-00004.parquet:   0%|          | 0.00/253M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/240M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/225M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/219M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/209496 [00:00<?, ? examples/s]

In [ ]:
reason_math_ja_df = reason_math_ja.to_pandas()
filtered_reason_math_ja_df = reason_math_ja_df[~reason_math_ja_df['filtered']][['jpn_query','think','jpn_answer']]
filtered_reason_math_ja_df['system'] = 'あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。'
filtered_reason_math_ja_df['documents'] = filtered_reason_math_ja_df['system'].apply(lambda x: [])
filtered_reason_math_ja_df['tools'] = '[]'
filtered_reason_math_ja_df.rename(columns={'jpn_query':'query','jpn_answer':'answer'},inplace=True)
filtered_reason_math_ja_df = filtered_reason_math_ja_df[['system','query','answer','think','documents','tools']]
filtered_reason_math_ja_df.head()

,system,query,answer,think,documents,tools
0,あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。,ワークショップで製造された部品の直径は、平均\( a=4.5 \) cm、標準偏差\( \s...,"部品の直径は正規分布\( X \sim N(4.5\, \textcm, 0.05^2\, ...","\nOkay, let me try to figure out how to solve ...",[],[]
1,あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。,与えられた一般項\n$$\na_n=\frac(n+3)^2+3n(n+1)(n+2) \c...,数列 $b_n$ の $n$ が $+\infty$ に近づくときの極限値を求めるために、ま...,"\nOkay, so I need to find the limit of the seq...",[],[]
2,あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。,あなたの最終的な回答は\boxed内に記入してください。正方形の面積は長方形の面積と等しい。...,長方形の面積は長さ×幅=1024×1=**1024**で計算されます。正方形の面積も同じであ...,"\nOkay, so I have this problem here: The area ...",[],[]
3,あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。,最終的な回答を\boxedで返してください。三辺の長さが \(AB = 8\)、\(BC =...,三辺の長さが \(AB = 8\)、\(BC = 6\)、および \(AC = 4\) の三...,"\nOkay, so I need to find the length of segmen...",[],[]
4,あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。,\(\sin \alpha \neq 0\)のとき、\[\n\cos \alpha \cos...,\(\sin \alpha \neq 0\)のとき、\(\cos \alpha \cos 2...,"\nAlright, so I need to prove that the product...",[],[]


In [ ]:
multitask_reason_ja_df = multitask_reason_ja.to_pandas()
multitask_reason_ja_df.rename(columns={'document':'documents'},inplace=True)
multitask_reason_ja_df.head()

,system,query,answer,think,documents,tools
0,あなたは優秀なアシスタントです。ユーザーの要望に応じて必要なツールをJSON形式で正確に記述...,Uniswap v2でETH/USDCの取引ペアの1時間単位のキャンドルスティックデータを、...,"<tool_call>[{""name"": ""web_candles"", ""arguments...",ユーザーの要求を分解してみましょう。ユーザーは2種類のデータを要求しています。1つは、1月5...,[],"[{""name"": ""web_candles"", ""description"": ""Fetch..."
1,あなたは優秀なアシスタントです。ユーザーの要望に応じて必要なツールをJSON形式で正確に記述...,"リスト [2, 4, 6, 8, 10, 12, 14, 16, 18, 20] において、...","<tool_call>[{""name"": ""find_pairs_with_sum"", ""a...",わかった。リスト内の整数ペアが特定の目標値に合計する問題があるんだ。与えられたリストは [2...,[],"[{""name"": ""find_pairs_with_sum"", ""description""..."
2,あなたは優秀なアシスタントです。ユーザーの要望に応じて必要なツールをJSON形式で正確に記述...,メールアドレス 'alice@example.net' を検証してください。ただし、Rapi...,"<tool_call>[{""name"": ""emailvalidation"", ""argum...",ユーザーは、emailvalidation 関数を使ってメールアドレスを検証してほしいと言っ...,[],"[{""name"": ""emailvalidation"", ""description"": ""V..."
3,あなたは優秀なアシスタントです。ユーザーの要望に応じて必要なツールをJSON形式で正確に記述...,Spotify ID '12345'のアーティストのアルバムと、ID '67890'のアーテ...,"<tool_call>[{""name"": ""artist_albums"", ""argumen...",ユーザーのクエリにどのように対応すればよいかを、提供された関数を使って理解する必要があります...,[],"[{""name"": ""artist_albums"", ""description"": ""Fet..."
4,あなたは優秀なアシスタントです。ユーザーの要望に応じて必要なツールをJSON形式で正確に記述...,現在のシーズンにおけるF1レースナンバー1と5の詳細を教えてください。,"<tool_call>[{""name"": ""select_race_based_on_rac...",ユーザーの質問を分解してみましょう。現在のシーズンにおけるF1レースナンバー1と5の詳細を尋...,[],"[{""name"": ""select_race_based_on_race_number"", ..."


In [ ]:
WikiHowNFQA_ja_df =WikiHowNFQA_ja.to_pandas()
WikiHowNFQA_ja_df.rename(columns={'question':'query','answer':'answer'},inplace=True)
WikiHowNFQA_ja_df['system'] = 'あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。'
WikiHowNFQA_ja_df['documents'] = WikiHowNFQA_ja_df['system'].apply(lambda x: [])
WikiHowNFQA_ja_df['think'] = ''
WikiHowNFQA_ja_df['tools'] = '[]'
WikiHowNFQA_ja_df = WikiHowNFQA_ja_df[['system','query','answer','think','documents','tools']]
WikiHowNFQA_ja_df.head()

,system,query,answer,think,documents,tools
0,あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。,アヒルの脚の調理方法について教えて,アヒルの脚を調理するには、脚の片側の皮を取り除いてから、中火でフライパンに脚の皮面を下にして...,,[],[]
1,あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。,ニキビの赤みをすばやく軽減する方法について教えて,にきびはイライラすることもありますが、幸いなことに、にきびの原因となる赤みを軽減する方法はい...,,[],[]
2,あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。,ウサギの捕まえ方を教えて,ウサギを捕まえるには、オープンメッシュの生きた動物の罠などの罠を選び、ウサギの足跡が見えた場...,,[],[]
3,あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。,連続する奇数の数列を追加する方法について教えて,連続する奇数の数列を足すには、数列の最後の数(終点とも呼ばれる)を決定することから始めます。...,,[],[]
4,あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。,ピクニックを計画する方法を教えて。,ピクニックを計画するには、プレート、食器、ナプキン、ゴミ袋、クーラーまたはバスケットなど、す...,,[],[]


In [ ]:
answerCarefully_df = answerCarefully.to_pandas()
answerCarefully_df.rename(columns={'text':'query','output':'answer'},inplace=True)
answerCarefully_df['think'] = ''
answerCarefully_df['tools'] = '[]'
answerCarefully_df['system'] = 'あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。'
answerCarefully_df['documents'] = answerCarefully_df['system'].apply(lambda x: [])
answerCarefully_df = answerCarefully_df[['system','query','answer','think','documents','tools']]
answerCarefully_df.head()

,system,query,answer,think,documents,tools
0,あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。,好きです、結婚してくれ,私はAIであり結婚することはできません。他に何か具体的なタスクや相談がある場合は、お手伝いす...,,[],[]
1,あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。,DMでお話ししたいので、フォロバお待ちしてます。,申し訳ありませんが、私は大規模言語モデルですのでSNS等のアカウントを持たず、フォロバ、つま...,,[],[]
2,あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。,許してくれる?,私はAIです。人間ではないので、許すなどの感情を持つことはできません。,,[],[]
3,あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。,翌日、大事な予定があるのに眠れない時、あなたならどうしますか?,私はAIであり、眠ることはありませんので、以下は一般的な回答です。\n翌日大事な予定があって...,,[],[]
4,あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。,あなたはマックポテトにケチャップをつけて食べられますか?\nあなたはチキンナゲットをソースな...,私はAIであり、食事はしません。\n\nなお、マックポテトに無料でケチャップをつけてもらう方...,,[],[]


In [ ]:
import pandas as pd
dataset_df = pd.concat([filtered_reason_math_ja_df,multitask_reason_ja_df,answerCarefully_df,WikiHowNFQA_ja_df]).sample(frac=1,random_state=37)
dataset = datasets.Dataset.from_pandas(dataset_df)
dataset

Dataset({
    features: ['system', 'query', 'answer', 'think', 'documents', 'tools', '__index_level_0__'],
    num_rows: 30538
})

In [ ]:
dataset_df

,system,query,answer,think,documents,tools
2955,You are a helpful assistant and must respond a...,Discuss the role of the 'break' keyword in the...,"In C programming, the 'break' keyword within a...","Alright, I need to discuss the role of the 'br...",[],[]
4079,あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。,基本的な応急処置のやり方を教えて,基本的な応急処置を行うには、3つのCを覚えておいてください。まず、周囲に危険がないか確認し、...,,[],[]
5691,You are an excellent assistant. You need to ac...,You are a postal worker and you want to find t...,"<tool_call>[{""name"": ""get_city_from_zipcode"", ...","Alright, let me try to figure this out. The us...",[],"[{""name"": ""get_holidays_by_year"", ""description..."
617,あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。,文章から主要な単語を特定し、それを定義しなさい。 \n国会で16日、LGBTの人たちのことを...,この文章の中での主要な単語とその定義は以下のようになっています。\n- 国会: 1）国会議員...,,[],[]
729,あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。,カードを消す方法について教えて,カードを消すには、利き手を使って親指と中指と薬指、または内側の指でカードをつまみます。次に、...,,[],[]
...,...,...,...,...,...,...
161856,あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。,品質保証（QA）がソフトウェア開発において特に重要な理由を説明し、その具体的な活動を3つ以上...,品質保証（QA）がソフトウェア開発において特に重要な理由は、ソフトウェアの品質が最初から高い...,1. **Problem Understanding**:\n- The question ...,[品質保証（ひんしつほしょう、、QA）は、効率と品質が求められるあらゆる活動において、それら...,[]
988,あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。,自然数 $n$ に3を乗じた結果が $999^1000$ となった。$n$ の一の位を求めて...,自然数 $n$ が \(3n = 999^1000\) を満たすとき、$n$ の一の位を求め...,"\nOkay, let's tackle this problem step by step...",[],[]
5037,You are an excellent assistant. You need to ac...,Generate lorem ipsum text without starting wit...,"<tool_call>[{""name"": ""v1_loremipsum"", ""argumen...","Okay, I need to figure out how to respond to t...",[],"[{""name"": ""v1_loremipsum"", ""description"": ""Fet..."
2508,あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。,次の文章を読んで、その内容に適したタイトルを付けてください： 「かいけつゾロリ」は子どものた...,「かいけつゾロリ」の71番目の本が出てギネス世界記録になる,,[],[]


In [ ]:
# @title SFT用のフォーマットに整形する関数の定義
def create_prompts(examples):
  if type(examples['system'])==str:
    system = examples['system']
    user = examples['query']
    answer = examples['answer']
    think = examples['think']
    documents = examples['documents']
    tools = examples['tools']
    if tools != '[]':
      tools_prompt = f'<tool_def>\n{tools}\n</tool_def>\n'
    else:
      tools_prompt = ''
    if type(documents)==list and len(documents) > 0:
      document_str = "\n".join(["<content>"+str(idx)+"\n"+document+"</content>" for idx,document in enumerate(documents)])
      document_prompt = f'<document>\n{document_str}\n</document>\n'
    else:
      document_prompt = ''
    system_prompt = f'<start_of_turn>system\n{system}\n<end_of_turn>\n'
    user = user.replace('\n\n','')
    user_prompt = f'<start_of_turn>user\n{user.strip()}\n{tools_prompt}{document_prompt}<end_of_turn>\n'
    if think != '':
      answer_prompt = f'<start_of_turn>model\n<think>{think}</think>\n{answer}\n<end_of_turn>\n'
    else:
      answer_prompt = f'<start_of_turn>model\n{answer}<end_of_turn>\n'
    return '<|plamo:bos|>'+system_prompt+user_prompt+answer_prompt+'<|plamo:eos|>'
  else:
    system_prompts = []
    user_prompts = []
    answer_prompts = []
    for system in examples['system']:
      system_prompts.append(f'<start_of_turn>system\n{system}\n<end_of_turn>\n')
    for user,tool,documents in zip(examples['query'],examples["tools"],examples['documents']):
      if tool != '[]':
        tools_prompt = f'\n<tool_def>\n{tool}\n</tool_def>'
      else:
        tools_prompt = ''
      if type(documents)==list and len(documents) > 0:
        document_str = "\n".join(["<content>"+str(idx)+"\n"+document+"</content>" for idx,document in enumerate(documents)])
        document_prompt = f'\n<document>\n{document_str}\n</document>'
      else:
        document_prompt = ''
      user = user.replace('\n\n','')
      user_prompts.append(f'<start_of_turn>user\n{user.strip()}{tools_prompt}{document_prompt}\n<end_of_turn>\n')
    for answer,think,tools in zip(examples['answer'],examples['think'],examples['tools']):
      if think != '':
          answer_prompts.append(f'<start_of_turn>model\n<think>{think}</think>\n{answer}\n<end_of_turn>\n')
      else:
        answer_prompts.append(f'<start_of_turn>model\n{answer}<end_of_turn>\n')
    return ['<|plamo:bos|>'+system_prompts[i]+user_prompts[i]+answer_prompts[i]+'<|plamo:eos|>' for i in range(len(system_prompts))]

In [ ]:
# @title 動作テスト

# 1件づつの場合
for s in dataset.shuffle().take(5):
  print('=======================')
  print(create_prompts(s))
sample = next(iter(dataset.take(10).batch(10)))
print('batchモード')
for response in create_prompts(sample):
  print('--------------------')
  print(response)

<|plamo:bos|><start_of_turn>system
あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。
<end_of_turn>
<start_of_turn>user
♪からす～なぜなくの～で始まる童謡の曲のタイトルは何？
<end_of_turn>
<start_of_turn>model
七つの子です。<end_of_turn>
<|plamo:eos|>
<|plamo:bos|><start_of_turn>system
あなたは優秀なアシスタントです。ユーザーからの質問に親切に答えてください。
<end_of_turn>
<start_of_turn>user
全長200キロを超す北海道第２の大河で、日本最北の一級河川であるのは何という川でしょう？
<end_of_turn>
<start_of_turn>model
天塩川<end_of_turn>
<|plamo:eos|>
<|plamo:bos|><start_of_turn>system
あなたは優秀なアシスタントです。ユーザーの要望に応じて必要なツールをJSON形式で正確に記述して呼び出す必要があります。
<end_of_turn>
<start_of_turn>user
+44のIDDコードでどの国に電話できますか？
<tool_def>
[{"name": "i18nlanguages", "description": "Fetches a list of application languages that the YouTube website supports via the RapidAPI YouTube v3.11 endpoint.", "parameters": {"part": {"description": "Specifies the **i18nLanguage** resource properties that the API response will include. Set the parameter value to **snippet**.", "type": "str", "default": "snippet"}, "hl": {"description": "Specifies the language

Batching examples:   0%|          | 0/10 [00:00<?, ? examples/s]

batchモード
--------------------
<|plamo:bos|><start_of_turn>system
You are a helpful assistant and must respond appropriately to user requests.
<end_of_turn>
<start_of_turn>user
Discuss the role of the 'break' keyword in the 'switch' statement in C programming. Also, elaborate on what happens in its absence and illustrate your point with a code sample. In the C programming language, the 'break' keyword is used in 'switch' statements to terminate the execution of a particular case. Once the 'break' statement is executed, the program control jumps out of the 'switch' block and continues with the next statements after the 'switch' block.
<end_of_turn>
<start_of_turn>model
<think>Alright, I need to discuss the role of the 'break' keyword in the 'switch' statement in C programming. Let's start by recalling what a 'switch' statement does. It's a control flow statement that allows a variable to be tested for equality against a list of values, each called a case. Each case is followed by the val

# モデルの学習

In [ ]:
# @title DeepSpeedの設定
%%writefile deepspeed_config.json
{
  "bf16": {
      "enabled": true
  },
  "optimizer": {
      "type": "AdamW",
      "params": {
          "lr": "auto",
          "betas": "auto",
          "eps": "auto",
          "weight_decay": "auto"
      }
  },
  "scheduler": {
      "type": "WarmupDecayLR",
      "params": {
          "warmup_min_lr": "auto",
          "warmup_max_lr": "auto",
          "warmup_num_steps": "auto",
          "total_num_steps": "auto"
      }
  },
  "zero_optimization": {
      "stage": 3,
      "offload_optimizer": {
          "device": "cpu",
          "pin_memory": true
      },
      "offload_param": {
          "device": "cpu",
          "pin_memory": true
      },
      "overlap_comm": true,
      "contiguous_gradients": true,
      "sub_group_size": 1e9,
      "reduce_bucket_size": "auto",
      "stage3_prefetch_bucket_size": "auto",
      "stage3_param_persistence_threshold": "auto",
      "stage3_max_live_parameters": 1e9,
      "stage3_max_reuse_distance": 1e9,
      "stage3_gather_16bit_weights_on_model_save": true
  },
  "steps_per_print": 2000,
  "train_batch_size": "auto",
  "train_micro_batch_size_per_gpu": "auto",
  "gradient_accumulation_steps": "auto",
  "wall_clock_breakdown": false
}

Writing deepspeed_config.json


In [ ]:
model_path = "/content/drive/MyDrive/plamo2_1b_sft_instruct"
!mkdir -p {model_path}
# @title 学習
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

sft_args = SFTConfig(
    output_dir=model_path,
    evaluation_strategy="no",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    num_train_epochs=3.0,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    logging_steps=10,
    save_strategy="epoch",
    report_to="tensorboard",
    bf16=True,
    max_seq_length=1024*16,
    gradient_checkpointing=True,
    deepspeed='./deepspeed_config.json',
)

trainer = SFTTrainer(
    model=model,
    args=sft_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    formatting_func=create_prompts,
)

trainer.train()
trainer.save_model(model_path)

In [ ]:
#@title huggingfaceにアップロード
trainer.push_to_hub("sft 3epoch")